In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
dir = "/content/drive/My Drive/Colab Notebooks/animal"

width = 299
height = 299

In [0]:
data_generator = ImageDataGenerator(
    rotation_range = 10,
    horizontal_flip = True,
    # vertical_flip = True,
    zoom_range = 0.1,
    # width_shift_range = 0.2,
    height_shift_range = 0.1,
    rescale = 1 / 255,
    validation_split = 0.2,
)

train_data = data_generator.flow_from_directory(
    dir,
    target_size = (width, height),
    batch_size = 8,
    # save_to_dir = '/content/drive/My Drive/Colab Notebooks/augment_animals',
    class_mode = 'categorical',
    subset = 'training'
)

validation_data = data_generator.flow_from_directory(
    dir,
    target_size = (width, height),
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'validation'
)

In [0]:
base_model = tf.keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    # classes=1000,
    classifier_activation="softmax",
)

In [0]:
model = tf.keras.models.Sequential([

    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax'),
])

In [0]:
base_model.summary()

In [0]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [0]:
model.compile(optimizer=SGD(), 
              loss = tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [0]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]

In [0]:
batch_size = 8

output = model.fit(
    train_data,
    steps_per_epoch = train_data.samples // batch_size,
    epochs = 10,
    validation_data = validation_data, 
    validation_steps = validation_data.samples // batch_size,    
    class_weight = {0:1 , 1:1 , 2:2},
    shuffle = True,
    callbacks=callbacks
)

In [0]:
plt.plot(output.history['accuracy'])
plt.show()

In [0]:
import cv2 as cv
img = cv.imread('image.png')
img = cv.resize(img, (299, 299))
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

img = img / 255.0

img = img.reshape(1, 299, 299, 3)

y_pred = model.predict(img)
print(y_pred)
print(np.argmax(y_pred))

In [0]:
model.save('animals.h5')

In [0]:
# load and test with tf only

"""
## Run inference on new data
Note that data augmentation and dropout are inactive at inference time.
"""

img = keras.preprocessing.image.load_img("PetImages/Cat/6779.jpg", target_size=image_size)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = predictions[0]
print(
    "This image is %.2f percent cat and %.2f percent dog."
    % (100 * (1 - score), 100 * score)
)